In [1]:
import torch
import requests
import numpy
import json
from torch.utils.data import Dataset, DataLoader
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.functional import one_hot
import matplotlib.pyplot as plt

class DatasetContrasenas(Dataset):

    def __init__(self, root = "train.txt", seq_len = 20) -> None:
        super(DatasetContrasenas, self).__init__()

        self.root = root
        self.seq_len = seq_len
        self.vocabs = build_vocab_from_iterator(self.get_vocab(), specials=["<start>", "'"], max_tokens=100)
        self.stoi = self.vocabs.get_stoi()
        self.itos = self.vocabs.get_itos()
        self.inputs = self.get_inputs()
        json.dump({"stoi": self.stoi, "itos": self.itos}, open("password_charmap.json", "w"))

    def get_vocab(self):
        fd = open(self.root, "r")
        for text in fd:
            yield [chars for chars in str(text).strip()]
        fd.close()

    def get_inputs(self):
        fd = open(self.root, "r")
        inputs = []
        for text in fd:
            text_data = [self.stoi[chars] for chars in str(text).strip()]
            text_data.insert(0, self.stoi["<start>"])
            while len(text_data) < self.seq_len:
                text_data.append(self.stoi["'"])
            inputs.append(text_data[:self.seq_len])
        inputs = numpy.array(inputs, dtype=numpy.int32)
        fd.close()
        return inputs

    def __getitem__(self, index):
        return self.inputs[index]

    def __len__(self):
        return len(self.inputs)

    def download_file_from_github(url, save_path):
            response = requests.get(url)

            if response.status_code == 200:
                with open(save_path, 'wb') as file:
                    file.write(response.content)
                print(f"File downloaded successfully and saved at: {save_path}")
            else:
                print(f"Failed to download file. Status code: {response.status_code}")

    def plot_error_rates(error_rates):
        epochs = np.arange(1, len(error_rates) + 1)

        # Plotting the error rates
        plt.plot(epochs, error_rates, marker='o', linestyle='-', color='b')

        # Adding labels and title
        plt.title('Error Rate During Training')
        plt.xlabel('Epochs')
        plt.ylabel('Error Rate')

        # Show the plot
        plt.grid(True)
        plt.show()



class ResBlock(torch.nn.Module):

    def __init__(self, in_channel):
        super(ResBlock, self).__init__()
        self.conv_layer = torch.nn.Sequential(
            torch.nn.LeakyReLU(0.02),
            torch.nn.Conv1d(in_channel, in_channel, 3, padding=1, bias=False),
            torch.nn.BatchNorm1d(in_channel),
            torch.nn.LeakyReLU(0.02),
            torch.nn.Conv1d(in_channel, in_channel, 3, padding=1, bias=False),
            torch.nn.BatchNorm1d(in_channel))

    def forward(self, inputs):
        outputs = self.conv_layer(inputs)
        return (outputs * 0.3) + inputs

class RedGeneradora(torch.nn.Module):

    def __init__(self, seq_len, vocab_len):
        super(RedGeneradora, self).__init__()
        self.seq_len = seq_len
        self.fc_layer = torch.nn.Sequential(
            torch.nn.Linear(128, 512, bias=False),
            torch.nn.LeakyReLU(0.02),
            torch.nn.Linear(512, 128 * seq_len, bias=False))

        self.conv_layer = torch.nn.Sequential(
            ResBlock(128),
            ResBlock(128),
            ResBlock(128),
            ResBlock(128),
            ResBlock(128),
            torch.nn.Conv1d(128, vocab_len, 3, padding=1, bias=False),
            torch.nn.Softmax(1))

    def forward(self, inputs):
        outputs = self.fc_layer(inputs)
        outputs = outputs.reshape(-1, 128, self.seq_len)
        outputs = self.conv_layer(outputs)
        outputs = outputs.permute(0, 2, 1)
        return outputs


class RedDiscriminadora(torch.nn.Module):

    def __init__(self, seq_len, vocab_len):
        super(RedDiscriminadora, self).__init__()
        self.conv_layer = torch.nn.Sequential(
            torch.nn.Conv1d(vocab_len, 128, 3, padding=1, bias=False),
            torch.nn.BatchNorm1d(128),
            torch.nn.ReLU(),
            ResBlock(128),
            ResBlock(128),
            ResBlock(128),
            ResBlock(128),
            ResBlock(128))

        self.fc_layer = torch.nn.Sequential(
            torch.nn.Flatten(),
            torch.nn.Linear(128 * seq_len, 512, bias=False),
            torch.nn.ReLU(),
            torch.nn.Linear(512, 1, bias=False))

    def forward(self, inputs):
        inputs = inputs.permute(0, 2, 1)
        outputs = self.conv_layer(inputs)
        outputs = self.fc_layer(outputs)
        return outputs



github_file_url = 'https://raw.githubusercontent.com/franciscoventurablancas/tesis/main/train.txt'
save_path = 'train.txt'

epochs = 20
lr = 0.0001
batch_size = 64
seq_len = 20
c = 0.01
n_c = 5

file = DatasetContrasenas.download_file_from_github(github_file_url, save_path)
dataset = DatasetContrasenas(seq_len=seq_len)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

vocab_size = len(dataset.stoi)
print(len(dataset))
print(vocab_size)

G = RedGeneradora(seq_len, vocab_size)
D = RedDiscriminadora(seq_len, vocab_size)

G = G.cuda()
D = D.cuda()

g_optim = torch.optim.Adam(params=G.parameters(), lr=lr, betas=(0.5, 0.9))
d_optim = torch.optim.Adam(params=D.parameters(), lr=lr, betas=(0.5, 0.9))


for epoch in range(epochs):
    for iters, inputs in enumerate(dataloader, 0):

        real_data = inputs.long().cuda()
        fake_data = torch.randn(batch_size, 128).float().cuda()

        real_data = one_hot(real_data, vocab_size).float()

        for _ in range(n_c):
            with torch.no_grad():
                generate_data = G(fake_data)
            d_real = D(real_data)
            d_fake = D(generate_data)
            d_real_loss = -torch.mean(d_real)
            d_fake_loss = torch.mean(d_fake)
            d_loss = d_fake_loss + d_real_loss

            d_optim.zero_grad()
            d_loss.backward()
            d_optim.step()

            for p in D.parameters():
                p.data = p.data.clamp(-c, c)

        fake_data = torch.randn(batch_size, 128).float().cuda()
        generate_data = G(fake_data)
        d_fake = D(generate_data)
        g_loss = -torch.mean(d_fake)

        g_optim.zero_grad()
        g_loss.backward()
        g_optim.step()

        if iters % 10 == 0:
            print("[+] Epoch: [%d/%d] G_Loss: %.4f D_Loss: %.4f" % (epoch+1, epochs, g_loss, d_loss))
            with torch.no_grad():
                sample_size = 5
                fake_data = torch.randn(sample_size, 128).float().cuda()
                generate_data = G(fake_data)
                generate_data = torch.distributions.Categorical(probs=generate_data)
                generate_data = generate_data.sample()
                text = ''
                for i in range(sample_size):
                    for j in range(seq_len):
                        text += dataset.itos[generate_data[i, j]]
                    text += "\n"
                print(text.rstrip("\n"))

G = G.cpu()
D = D.cpu()

torch.save(G.state_dict(), "passgan_modelG.pth")
torch.save(D.state_dict(), "passgan_modelD.pth")

#En la siguiente sección se abre el nuevo archivo donde se depositaron las contraseñas
fd = open(f"generarContrasenas2.txt", "w", encoding='utf-8')
itos = json.load(open("./password_charmap.json", "r"))['itos']
model = RedGeneradora(seq_len, vocab_size)
model.load_state_dict(torch.load("./passgan_modelG.pth"))
model.eval()

sample_size = 100000
batch_size = 200
current_size = 0
seq_len = 20

while current_size < sample_size:
    fake_data = torch.randn(batch_size, 128).float()
    generate_data = G(fake_data)
    generate_data = torch.distributions.Categorical(probs=generate_data)
    generate_data = generate_data.sample()
    for i in range(batch_size):
        text = ""
        for j in range(seq_len):
            if itos[generate_data[i, j]] == "'":
                break
            text += itos[generate_data[i, j]]
        fd.write(text + "\n")
    current_size += batch_size
    print("[+] Generate Data: %d" % current_size)
fd.close()

















/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)


File downloaded successfully and saved at: train.txt
1000000
96


/usr/local/lib/python3.10/dist-packages/torch/autograd/graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Se han truncado las últimas 5000 líneas del flujo de salida.
[+] Epoch: [1/1] G_Loss: 1.0247 D_Loss: -0.0190
<start>zl7323452''''''''''
<start>auconk232''''''''''
<start>MARIS'''''''j''''''
<start>peetegillith'''''''
<start>dfifave8'''''''''''
[+] Epoch: [1/1] G_Loss: 1.0004 D_Loss: -2.1761
<start>19282548'''''''''''
<start>tarrgy'''''''''''''
<start>boll''''''''4''''''
<start>adcaml'9''''a''''''
<start>8SDw_iacyouill'''''
[+] Epoch: [1/1] G_Loss: 1.0214 D_Loss: -0.1151
<start>jrebolat'0seels''''
0112325''''''b2'vr'p
<start>ROrTA560216ixl7n'''
<start>bilomher''g''l'''''
<start>jaraykaxi'''a''''''
[+] Epoch: [1/1] G_Loss: 1.0214 D_Loss: -0.0241
<start>mngimidr'''m''''m''
<start>stzMc''''''o'''''''
<start>sjilovenca'''l'''''
<start>1wzA21432100'bver1'
<start>choyosj37''''''''''
[+] Epoch: [1/1] G_Loss: 1.0071 D_Loss: -0.0194
<start>shenuvaly''''''''''
<start>360fobdtyh3''''''''
<start>2112031'7'''''''3''
<start>cooli''''''''''''''
<start>bultarer'''''''''''
[+] Epoch: [1/1] G_Loss: 0.991